In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    "cifar10",
    split=["train","test"],
    shuffle_files=True,
    as_supervised=True,# will return turple(img, label) otherwise dict
    with_info=True# able to get ifro about dataset
)

def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label





c:\Users\Lenovo\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
for image, label in ds_train.take(1):
    print("Image shape:", image.shape)
    print("Label:", label)


Image shape: (32, 32, 3)
Label: tf.Tensor(7, shape=(), dtype=int64)


In [3]:
AUTOTUNE = tf.data.AUTOTUNE 
BATCH_SIZE = 32


def augment(image, label):
    new_height = new_width = 32
    image = tf.image.resize(image, (new_height, new_width))
    
    # if tf.random.uniform((), minval=0, maxval=1) < 0.1:
        # image= tf.tile(tf.image.rgb_to_grayscale(image), [1, 1, 3])
    
    image = tf.image.random_brightness(image, max_delta=0.1)
    
    image = tf.image.random_contrast(image, lower=0.1, upper=0.2)
    
    image = tf.image.random_flip_left_right(image) #50%
    #image = tf.image.random_flip_up_down(image) #50%
    
    return image, label


ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)# другий параметр щоб паралельно запустити
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
# ds_train = ds_train.map(augment, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

        

# Setup for test Dataset
ds_test = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_train.batch(BATCH_SIZE)
ds_test = ds_train.prefetch(AUTOTUNE)



In [4]:
# print(ds_train.shape)
for image, label in ds_train.take(1):
    print(image.shape)  # Виводить форму зображення
    print(label.shape)  # Виводить форму мітки (label)


(32, 32, 32, 3)
(32,)


In [5]:
model = keras.Sequential(
    [
        keras.layers.InputLayer(input_shape=(32, 32, 3)),  # specify the input shape
        layers.Conv2D(4, 3, padding="same", activation="relu"),
        layers.Conv2D(8, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(16, 3, padding="same", activation="relu"),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(10)
    ]
)

In [6]:
model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)


model.fit(ds_train, epochs=5, verbose=2)
model.evaluate(ds_test)


Epoch 1/5
1563/1563 - 12s - loss: 1.6907 - accuracy: 0.3853 - 12s/epoch - 8ms/step
Epoch 2/5
1563/1563 - 5s - loss: 1.4070 - accuracy: 0.4962 - 5s/epoch - 3ms/step
Epoch 3/5
1563/1563 - 5s - loss: 1.3036 - accuracy: 0.5390 - 5s/epoch - 3ms/step
Epoch 4/5
1563/1563 - 5s - loss: 1.2352 - accuracy: 0.5664 - 5s/epoch - 3ms/step
Epoch 5/5
1563/1563 - 5s - loss: 1.1836 - accuracy: 0.5863 - 5s/epoch - 3ms/step
1563/1563 [==============================] - 4s 3ms/step - loss: 1.1285 - accuracy: 0.6067


[1.1285114288330078, 0.6067399978637695]

In [7]:
# import os

# os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    "cifar10",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,  # will return tuple (img, label) otherwise dict
    with_info=True,  # able to get info about dataset
)


def normalize_img(image, label):
    """Normalizes images"""
    return tf.cast(image, tf.float32) / 255.0, label


AUTOTUNE = tf.data.AUTOTUNE 
BATCH_SIZE = 32


def augment(image, label):
    new_height = new_width = 32
    image = tf.image.resize(image, (new_height, new_width))

    if tf.random.uniform((), minval=0, maxval=1) < 0.1:
        image = tf.tile(tf.image.rgb_to_grayscale(image), [1, 1, 3])

    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.1, upper=0.2)

    # a left upside down flipped is still a dog ;)
    image = tf.image.random_flip_left_right(image)  # 50%
    # image = tf.image.random_flip_up_down(image) #%50%

    return image, label



# Setup for test Dataset
ds_test = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_train.batch(BATCH_SIZE)
ds_test = ds_train.prefetch(AUTOTUNE)

# TF >= 2.3.0
data_augmentation = keras.Sequential(
    [
        tf.keras.layers.Resizing(height=32, width=32,),
        tf.keras.layers.RandomFlip(mode="horizontal"),
        tf.keras.layers.RandomContrast(factor=0.1,),
    ]
)

model = keras.Sequential(
    [
        keras.Input((32, 32, 3)),
        data_augmentation,
        layers.Conv2D(4, 3, padding="same", activation="relu"),
        layers.Conv2D(8, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(16, 3, padding="same", activation="relu"),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(10),
    ]
)

model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.fit(ds_train, epochs=5, verbose=2)
model.evaluate(ds_test)

Epoch 1/5


ValueError: in user code:

    File "c:\Users\Lenovo\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Lenovo\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Lenovo\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Lenovo\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Lenovo\anaconda3\envs\tf\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Lenovo\anaconda3\envs\tf\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibilit